# Text document Classification (Dataset)

https://www.kaggle.com/datasets/jensenbaxter/10dataset-text-document-classification

*   10 clases
*   1000 muestras de entrenamiento

In [2]:
import zipfile
with zipfile.ZipFile('/content/Dataset_text.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Dataset_text/')

In [3]:
from tensorflow import keras
batch_size = 32

In [ ]:
"""
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(2023).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

In [4]:
train_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="training",
)

Found 1000 files belonging to 10 classes.
Using 800 files for training.


In [5]:
val_ds = keras.utils.text_dataset_from_directory(
    "/content/Dataset_text",
    batch_size=batch_size,
    validation_split=0.2,
    label_mode="categorical",
    seed = 2023,
    subset="validation",
)

Found 1000 files belonging to 10 classes.
Using 200 files for validation.


In [ ]:
"""
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size
)
"""

'\ntest_ds = keras.utils.text_dataset_from_directory(\n    "aclImdb/test",\n    batch_size=batch_size\n)\n'

In [6]:
# Visualizar características del primer lote (batch)
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32, 10)
targets.dtype: <dtype: 'float32'>
inputs[0]: tf.Tensor(b'Toward Armistice\r\nBy the fall of 1918, the Central Powers were unraveling on all fronts.\r\n\r\nDespite the Turkish victory at Gallipoli, later defeats by invading forces and an Arab revolt had combined to destroy the Ottoman economy and devastate its land, and the Turks signed a treaty with the Allies in late October 1918.\r\n\r\nAustria-Hungary, dissolving from within due to growing nationalist movements among its diverse population, reached an armistice on November 4. Facing dwindling resources on the battlefield, discontent on the homefront and the surrender of its allies, Germany was finally forced to seek an armistice on November 11, 1918, ending World War I.', shape=(), dtype=string)
targets[0]: tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0. 0. 0.], shape=(10,), dtype=float32)


# Data preparation



In [7]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# No se utilizan ngrams (none)
max_length = 600
max_tokens = 20000
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,    # Limitar la entrada a 600 palabras
)

# Extraer solo el texto (features) de los datos de entrada para calcular el vocabulario
text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization.adapt(text_only_train_ds)

"""
int_train_ds = train_ds.map(
    lambda x, y: (tf.one_hot(text_vectorization(x), max_tokens), y),
    num_parallel_calls=4)

int_val_ds = val_ds.map(
    lambda x, y: (tf.one_hot(text_vectorization(x), max_tokens), y),
    num_parallel_calls=4)

int_test_ds = test_ds.map(
    lambda x, y: (tf.one_hot(text_vectorization(x), max_tokens), y),
    num_parallel_calls=4)
"""



int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

"""
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
"""

'\nint_test_ds = test_ds.map(\n    lambda x, y: (text_vectorization(x), y),\n    num_parallel_calls=4)\n'

# Modelo

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Dropout, Bidirectional

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)

model_Embedded = keras.Model(inputs, outputs)

model_Embedded.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model_Embedded.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 bidirectional (Bidirection  (None, 64)                73984     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
Total params: 5194634 (19.82 MB)
Trainable params: 5194634 (19.82 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

# Entrenamiento

In [ ]:
# Utilización de caché
model_Embedded.fit(
    int_train_ds,
    validation_data=int_val_ds,
    epochs=20,
    #callbacks=callbacks
    )

Epoch 1/20
25/25 [==============================] - 14s 224ms/step - loss: 2.2840 - accuracy: 0.1500 - val_loss: 2.2549 - val_accuracy: 0.3050
Epoch 2/20
25/25 [==============================] - 5s 197ms/step - loss: 2.1560 - accuracy: 0.2912 - val_loss: 2.0920 - val_accuracy: 0.2400
Epoch 3/20
25/25 [==============================] - 5s 189ms/step - loss: 1.9073 - accuracy: 0.3038 - val_loss: 1.9141 - val_accuracy: 0.3400
Epoch 4/20
25/25 [==============================] - 4s 159ms/step - loss: 1.7533 - accuracy: 0.3562 - val_loss: 1.7588 - val_accuracy: 0.4150
Epoch 5/20
25/25 [==============================] - 5s 196ms/step - loss: 1.5786 - accuracy: 0.4475 - val_loss: 1.5984 - val_accuracy: 0.4950
Epoch 6/20
25/25 [==============================] - 4s 167ms/step - loss: 1.3721 - accuracy: 0.5537 - val_loss: 1.4669 - val_accuracy: 0.5300
Epoch 7/20
25/25 [==============================] - 4s 175ms/step - loss: 1.2241 - accuracy: 0.6000 - val_loss: 1.4043 - val_accuracy: 0.5900
Epoch

#Evaluación del modelo

Sobre datos test